<a href="https://colab.research.google.com/github/YBharatiyadav/CBT-CIP/blob/main/ChatBot_Using_RAG_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install -U langchain-community
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader

In [26]:
!mkdir -p models
!wget -O models/llama-2-7b-chat.Q4_K_M.gguf https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf


--2025-03-23 08:15:36--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.24, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/08a5566d61d7cb6b420c3e4387a39e0078e1f2fe5f055f3a03887385304d4bfa?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q4_K_M.gguf%3B+filename%3D%22llama-2-7b-chat.Q4_K_M.gguf%22%3B&Expires=1742721336&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjcyMTMzNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlLzA4YTU1NjZkNjFkN2NiNmI0MjBjM2U0Mzg3YTM5ZTAwNzhlMWYyZmU1ZjA1NWYzYTAzODg

## Check how it is work?

In [27]:
!pip install llama-cpp-python

from llama_cpp import Llama


model_path = "models/llama-2-7b-chat.Q4_K_M.gguf"
llm = Llama(model_path=model_path, n_ctx=2048)
prompt = "Hello, how are you?"
response = llm(prompt, max_tokens=150)
print(response["choices"][0]["text"])

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32         

 My name is Sarah and I'm a 3rd year PhD student in the Department of Psychology at the University of York. My research focuses on the cognitive and neural mechanisms underlying social cognition, with a particular emphasis on empathy and moral decision-making.
I'm currently conducting a study on the neural basis of empathy and would love to speak with you about your experiences and thoughts on this topic. The study should only take about 30 minutes of your time and can be conducted online or in person, whichever you prefer.
Would you be willing to participate? If so, please let me know your availability and we can schedule a time that works for


## Brain Stroke Information Chatbot


*   Answers stroke-related queries using ChromaDB & LLM








In [8]:
import os
import chromadb
import pandas as pd
!pip install streamlit
import streamlit as st
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from llama_cpp import Llama

# Load the embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="chroma_db")
collection = chroma_client.get_or_create_collection(name="stroke_docs")
chat_history_collection = chroma_client.get_or_create_collection(name="chat_history")

# Load and process the PDF file with batch embeddings
def process_pdf(pdf_path="/content/Preventing-Stroke-Presentation.pdf"):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # Chunking the text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_documents(pages)

    # Extract only text for batch embedding
    chunk_texts = [chunk.page_content for chunk in chunks]

    # Generate embeddings in batch
    embeddings = embedder.encode(chunk_texts, batch_size=4, show_progress_bar=True)

    # Store in ChromaDB
    for idx, (chunk_text, embedding) in enumerate(zip(chunk_texts, embeddings)):
        collection.add(ids=[f"doc_{idx}"], documents=[chunk_text], embeddings=[embedding.tolist()])

    print("✅ New PDF successfully processed and stored in ChromaDB!")

# Process the PDF and store in ChromaDB
process_pdf()

# Load GGUF model
model_path = "./models/llama-2-7b-chat.Q4_K_M.gguf"
llm = Llama(model_path=model_path, n_ctx=2048)

# Query processing
def retrieve_relevant_chunks(query):
    query_embedding = embedder.encode([query])[0].tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=3)
    return results["documents"][0] if "documents" in results else []

# Store and retrieve chat history
def store_chat_history(user_query, bot_response):
    chat_text = f"User: {user_query}\nAssistant: {bot_response}"
    chat_embedding = embedder.encode([chat_text])[0].tolist()
    chat_docs = chat_history_collection.peek(1)  # Retrieve at least one stored document
    chat_count = len(chat_docs["documents"]) if chat_docs and "documents" in chat_docs else 0
    chat_history_collection.add(ids=[f"chat_{chat_count}"], documents=[chat_text], embeddings=[chat_embedding])

def retrieve_chat_history():
    results = chat_history_collection.peek(5)  # Retrieve last 5 chat interactions
    return "\n".join([doc for doc in results["documents"]]) if "documents" in results else ""

# Generate response using GGUF model
def generate_response(query, context=""):
    chat_history = retrieve_chat_history()
    prompt = f"{chat_history}\nContext: {context}\nUser: {query}\nAssistant:"
    response = llm(prompt, max_tokens=256)
    short_responses = {
        "types of stroke": "Ischemic, Hemorrhagic, TIA",
        "what is a stroke": "A stroke occurs when blood flow to the brain is blocked or a vessel bursts."
    }
    for key, value in short_responses.items():
        if key in query.lower():
            return value
    return response["choices"][0]["text"].strip()

# Streamlit UI
st.title("🧠 Stroke Information Chatbot")
st.write("Ask me anything about strokes below:")

user_query = st.text_input("Enter your question:")
if user_query:
    context_chunks = retrieve_relevant_chunks(user_query)
    context_text = " ".join(context_chunks)
    response = generate_response(user_query, context_text)
    store_chat_history(user_query, response)
    st.write(f"**Chatbot:** {response}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32       

✅ New PDF successfully processed and stored in ChromaDB!


load: control token:      2 '</s>' is not marked as EOG
load: control token:      1 '<s>' is not marked as EOG
load: special_eos_id is not in special_eog_ids - the tokenizer config may be incorrect
load: special tokens cache size = 3
load: token to piece cache size = 0.1684 MB
print_info: arch             = llama
print_info: vocab_only       = 0
print_info: n_ctx_train      = 4096
print_info: n_embd           = 4096
print_info: n_layer          = 32
print_info: n_head           = 32
print_info: n_head_kv        = 32
print_info: n_rot            = 128
print_info: n_swa            = 0
print_info: n_embd_head_k    = 128
print_info: n_embd_head_v    = 128
print_info: n_gqa            = 1
print_info: n_embd_k_gqa     = 4096
print_info: n_embd_v_gqa     = 4096
print_info: f_norm_eps       = 0.0e+00
print_info: f_norm_rms_eps   = 1.0e-06
print_info: f_clamp_kqv      = 0.0e+00
print_info: f_max_alibi_bias = 0.0e+00
print_info: f_logit_scale    = 0.0e+00
print_info: f_attn_scale     = 0.0e+00
p